In [1]:
import os
import re
import sys
sys.path.append(os.getcwd() + "/..")
import numpy as np
import pandas as pd
import pickle as pkl
from osgeo import gdal
from FunctionalCode.StatsPlotFuncs import StatsPlot

In [ ]:
### 1.绘制训练数据的直方图
# * 训练数据配置
# input_dir = "/fossfs/skguan/data_fusion/landsat"
# dataset_file = "/fossfs/skguan/data_fusion/dataset.csv"
# df = pd.read_csv(dataset_file)
# train_files = df['train'].dropna().astype(str).tolist()
# train_files += df['val'].dropna().astype(str).tolist()
# test_files = df['test'].dropna().astype(str).tolist()
# files = test_files  # TODO: 这里修改需要绘制的数据
# site_names = [x.split("/")[-2] for x in files][10:20]
# data = np.empty(shape=(7, 0, 256, 256))
# for i, site_name in enumerate(site_names):
    # input_path = os.path.join(input_dir, site_name)
    # input_file = os.path.join(input_path, site_name + ".npy")
    # imgs = np.load(input_file)
    # data = np.concatenate((data, imgs), axis=1)

# * Land Cover 配置
map_dict = {
    10: [10, 20],  # Croplands
    11: [11, 12],  # Cropland / Natural Vegetation Mosaics
    51: [51, 52],  # Evergreen Broadleaf Forests
    61: [61, 62],  # Deciduous Broadleaf Forests
    71: [71, 72],  # Evergreen Needleleaf Forests
    81: [81, 82],  # Deciduous Needleleaf Forests
    91: [91, 92],  # Mixed Forests
    120: [120, 121, 122],  # Shrublands
    130: [130],  # Grasslands
    150: [140, 150, 152, 153],  # Sparse Vegetation
    181: [181, 182, 184, 185, 186, 187],  # Permanent Wetlands
    190: [190],  # Impervious Surfaces
    200: [200, 201, 202],  # Bare Areas
    210: [210],  # Water Body
    220: [220],  # Snow and Ice
}

input_dir = "/fossfs/skguan/data_fusion/glc"
input_files = list(map(
    lambda x: os.path.join(input_dir, x),
    os.listdir(input_dir)
))
data = np.empty(shape=(0, 257, 257))
for input_file in input_files:
    imgs = gdal.Open(input_file).ReadAsArray()
    values = []
    for k, v in map_dict.items():
        imgs = np.where(np.isin(imgs, v), k, imgs)
        values.append(k)
    imgs = np.where(np.isin(imgs, values), imgs, 0)
    data = np.concatenate((data, imgs[None, :, :]), axis=0)

data = data.reshape(-1,)
sp = StatsPlot()
sp.hist_plot(data)

In [ ]:
### 2.绘制训练日志里面的精度曲线
metrics = {
    "train_r2": [], "train_rmse": [], "train_psnr": [], "train_ssim": [],
    "valid_r2": [], "valid_rmse": [], "valid_psnr": [], "valid_ssim": []
}
pattern = ".+, (?P<metric>\\w+)_r2: (?P<r2>-?\\d+\\.?\\d+), \\w+_rmse: (?P<rmse>\\d+\\.?\\d+), " \
    "\\w+_psnr: (?P<psnr>\\d+\\.?\\d+), \\w+_ssim: (?P<ssim>\\d+\\.?\\d+)"
with open("./logs/training_20250620_vit(LMF)_2148206.out", 'r') as f:
    lines = f.readlines()
    for line in lines:
        if "train_r2" in line or "valid_r2" in line:
            regex = re.match(pattern, line)
            metrics["%s_r2" % regex.group("metric")].append(float(regex.group("r2")))
            metrics["%s_rmse" % regex.group("metric")].append(float(regex.group("rmse")))
            metrics["%s_psnr" % regex.group("metric")].append(float(regex.group("psnr")))
            metrics["%s_ssim" % regex.group("metric")].append(float(regex.group("ssim")))

# with open("metrics copy.pkl", 'rb') as f:
#     metrics = pkl.load(f)

key = ["r2", "rmse", "psnr", "ssim"]
output_name = "metric_plot_vit(LMF)"
sp = StatsPlot()
sp.line_plot(metrics, key, col=2)

In [8]:
### 3.保存预测的数据
data = np.load("/fossfs/skguan/output/data_fusion/val_files/val_result_fold0_SwinSTFM.npz")
label = data['label'].reshape(-1, 6, 256, 256)[3, :, :, :]
pred = data['pred'].reshape(-1, 6, 256, 256)[3, :, :, :]

from osgeo import gdal
def save_image(output_path, data):
    driver = gdal.GetDriverByName("GTiff")
    if 'int8' in data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32

    if len(data.shape) == 3:
        band_num = data.shape[0]
        width = data.shape[1]
        length = data.shape[2]
    else:
        band_num = 1
        width = data.shape[0]
        length = data.shape[1]
    out_ds = driver.Create(
        output_path,
        length,
        width,
        band_num,
        datatype
    )

    if band_num > 1:
        for i in range(band_num):
            out_ds.GetRasterBand(i + 1).WriteArray(data[i])
    else:
        out_ds.GetRasterBand(1).WriteArray(data)

    del out_ds

save_image("label.tif", label)
save_image("pred.tif", pred)

: 

In [ ]:
# ### 4. 密度散点图可视化
# data_path = "/home/gskk/Code/data_fusion/test_result_fold0_vit(SEASON_2Sat).npz"
# labels = ["Deep Blue", "Blue", "Green", "Red", "NIR", "SWIR-1", "SWIR-2"]
# data = np.load(data_path)
# label_data = data['label']
# pred_data = data['pred']
# sp = StatsPlot()
# for i, label_name in enumerate(labels):
#     label = label_data[:, i, :, :].reshape(-1,)
#     pred = pred_data[:, i, :, :].reshape(-1,)
#     file_name =  f"vit(season_2sat)_{label_name}.png"
#     sp.point_scatter_density(label, pred, file_name=file_name)